In [16]:
import numpy as np
import ikfastpy
import math
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

In [17]:
import numpy as np
import ikfastpy
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

# Initialize kinematics for UR5 robot arm
ur5_kin = ikfastpy.PyKinematics()
n_joints = ur5_kin.getDOF()

joint_angles = [-3.1, -1.6, 1.6, -1.6, -1.6, 0.0] # in radians

# Test forward kinematics: get end effector pose from joint angles
print("\nTesting forward kinematics:\n")
print("Joint angles:")
print(joint_angles)
ee_pose = ur5_kin.forward(joint_angles)
ee_pose = np.asarray(ee_pose).reshape(3,4) # 3x4 rigid transformation matrix
print("\nEnd effector pose:")
print(ee_pose)
print("\n-----------------------------")

# Test inverse kinematics: get joint angles from end effector pose
print("\nTesting inverse kinematics:\n")
joint_configs = ur5_kin.inverse(ee_pose.reshape(-1).tolist())
n_solutions = int(len(joint_configs)/n_joints)
print("%d solutions found:"%(n_solutions))
joint_configs = np.asarray(joint_configs).reshape(n_solutions,n_joints)
for joint_config in joint_configs:
    print(joint_config)

# Check cycle-consistency of forward and inverse kinematics
assert(np.any([np.sum(np.abs(joint_config-np.asarray(joint_angles))) < 1e-4 for joint_config in joint_configs]))
print("\nTest passed!")


Testing forward kinematics:

Joint angles:
[-3.1, -1.6, 1.6, -1.6, -1.6, 0.0]

End effector pose:
[[0.039 -0.999 0.030 0.477]
 [-0.999 -0.040 -0.028 0.150]
 [0.029 -0.029 -0.999 0.491]]

-----------------------------

Testing inverse kinematics:

8 solutions found:
[0.587 -3.054 1.595 3.039 1.531 0.545]
[0.587 -2.974 0.981 0.432 -1.531 -2.596]
[0.587 -2.036 -0.981 1.456 -1.531 -2.596]
[-3.100 -1.600 1.600 -1.600 -1.600 0.000]
[0.587 -1.542 -1.595 -1.566 1.531 0.545]
[-3.100 -1.105 0.975 1.672 1.600 -3.142]
[-3.100 -0.083 -1.600 0.083 -1.600 0.000]
[-3.100 -0.173 -0.975 2.690 1.600 -3.142]

Test passed!


In [15]:
conversion(solUR5ik(ee_pose[:,3:],ee_pose[:,0:3]))

array([[-3.100, -1.106, 0.976, 1.672, 1.600, -3.140],
       [-3.100, -0.173, -0.976, 2.690, 1.600, -3.140],
       [-3.100, -1.599, 1.599, -1.600, -1.600, 0.001],
       [-3.100, -0.083, -1.599, 0.082, -1.600, 0.001],
       [0.583, -3.054, 1.594, 3.041, 1.531, 0.544],
       [0.583, -1.542, -1.594, -1.566, 1.531, 0.544],
       [0.583, -2.974, 0.982, 0.431, -1.531, -2.598],
       [0.583, -2.035, -0.982, 1.456, -1.531, -2.598]])

In [18]:
def forwardkinematics(thetas):

    d1 = 0.1625
    a2 = -0.425
    a3 = -0.3922
    d4 = 0.1333
    d5 = 0.0997 
    d6 = 0.0996 
    t1 = thetas[0]
    t2 = thetas[1]
    t3 = thetas[2]
    t4 = thetas[3]
    t5 = thetas[4]
    t6 = thetas[5]

    dh = np.array([[0, 0, d1, t1],
                  [np.deg2rad(90), 0, 0, t2],
                  [0, 0, d4, 0],
                  [0, a2, 0, t3],
                  [0, 0, -d4, 0],
                  [0, a3, 0, 0],
                  [0, 0, d4, t4],
                  [np.deg2rad(90), 0, d5, t5],
                  [np.deg2rad(-90), 0, d6, t6]])
    
    T = []
    for i in range(len(dh)):
        t = np.array([[np.cos(dh[i][3]), -np.sin(dh[i][3]), 0, dh[i][1]],
                      [np.sin(dh[i][3])*np.cos(dh[i][0]), np.cos(dh[i][3])*np.cos(dh[i][0]), -np.sin(dh[i][0]), (-np.sin(dh[i][0])*dh[i][2])],
                      [np.sin(dh[i][3])*np.sin(dh[i][0]), np.cos(dh[i][3])*np.sin(dh[i][0]), np.cos(dh[i][0]), np.cos(dh[i][0])*dh[i][2]],
                      [0, 0, 0, 1]])
        T.append(t)
    return T

T = forwardkinematics(joint_angles)
t = [T[0]]
for i in range(len(T)-1):
    t.append(np.matmul(t[i],T[i+1]))

FK_pose = t[-1][:3,:4]
FK_pose

array([[0.041, -0.999, 0.030, 0.477],
       [-0.999, -0.042, -0.028, 0.150],
       [0.029, -0.029, -0.999, 0.491]])

In [19]:
ee_pose

array([[0.039, -0.999, 0.030, 0.477],
       [-0.999, -0.040, -0.028, 0.150],
       [0.029, -0.029, -0.999, 0.491]])

In [12]:
R = FK_pose[:,0:3]
R

array([[0.041, -0.999, 0.030],
       [-0.999, -0.042, -0.028],
       [0.029, -0.029, -0.999]])

In [13]:
p = FK_pose[:,3:]
p

array([[0.477],
       [0.150],
       [0.491]])

In [14]:
conversion(solUR5ik(p,R))

array([[-3.100, -1.107, 0.978, 1.671, 1.600, -3.142],
       [-3.100, -0.172, -0.978, 2.691, 1.600, -3.142],
       [-3.100, -1.600, 1.600, -1.600, -1.600, 0.000],
       [-3.100, -0.083, -1.600, 0.083, -1.600, 0.000],
       [0.584, -3.054, 1.595, 3.040, 1.531, 0.543],
       [0.584, -1.542, -1.595, -1.566, 1.531, 0.543],
       [0.584, -2.975, 0.983, 0.430, -1.531, -2.599],
       [0.584, -2.034, -0.983, 1.457, -1.531, -2.599]])

In [8]:
#inverse kinematics
def DHTable2HomTrans(DHTable):
    T = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]
    for i in range(np.shape(DHTable)[0]):
        T1 = np.dot(T,DH2HomTrans(DHTable[i]))
        T = T1
    return T

def DH2HomTrans(DHparams):
    [al,a,d,th] = DHparams
    T = [[np.cos(th),-np.sin(th)*np.cos(al),np.sin(th)*np.sin(al),a*np.cos(th)],[np.sin(th),np.cos(th)*np.cos(al),-np.cos(th)*np.sin(al),a*np.sin(th)],[0,np.sin(al),np.cos(al),d],[0,0,0,1]]
    return T

def sol2Rik(x,y,l1,l2):
    a = -2.0*l1*x
    b = -2.0*l1*y
    c = l1**2.0 - l2**2.0 + x**2.0 + y**2.0
    d = -c/np.sqrt(a**2.0+b**2.0)
    theta1 = [np.arctan2(b,a)+np.arccos(d),np.arctan2(b,a)-np.arccos(d)]
    theta2 = [None]*len(theta1)
    j = 0
    for i in theta1:
        theta12 = np.arctan2((y - l1*np.sin(i))/l2,(x - l1*np.cos(i))/l2)
        theta2[j] = theta12 - i
        j = j+1
    t1t2 = np.column_stack((theta1, theta2))
    return t1t2

def solUR5ik(p,R):
    d1 = 0.1625
    a2 = -0.425
    a3 = -0.3922
    d4 = 0.1333
    d5 = 0.0997 #+ 0.025
    d6 = 0.0996 #+ 0.08165
    [x,y,z] = p
    [[nx,ox,ax],[ny,oy,ay],[nz,oz,az]] = R
    a = ay*d6 - y
    b = -ax*d6 + x
    c = -d4
    d = -c/np.sqrt(a**2.0+b**2.0)
    theta1 = [np.arctan2(b,a)+np.arccos(d),np.arctan2(b,a)-np.arccos(d)]
    theta5 = [None]*len(theta1)
    j = 0
    for i in theta1:
        theta5[j] = [np.arccos((-d4-y*np.cos(i)+x*np.sin(i))/d6),-np.arccos((-d4-y*np.cos(i)+x*np.sin(i))/d6)]
        j = j+1
    
    
    t1t5 = [[theta1[0],theta5[0][0]],[theta1[0],theta5[0][1]],[theta1[1],theta5[1][0]],[theta1[1],theta5[1][1]]]
    theta6 = [None]*np.shape(t1t5)[0]
    j = 0
    for i in range(np.shape(t1t5)[0]):
        theta6[j] = [np.arctan2((oy*np.cos(t1t5[i][0])-ox*np.sin(t1t5[i][0]))/np.sin(t1t5[i][1]),(-ny*np.cos(t1t5[i][0])+nx*np.sin(t1t5[i][0]))/np.sin(t1t5[i][1]))]
        j = j+1     
    t1t5t6 = np.hstack((t1t5,theta6))
    k = 0
    t2t3t4 = [None]*np.shape(t1t5)[0]*2  
    TUR5 = np.vstack((np.hstack((R,np.array(p).reshape(3,1))),[0,0,0,1]))
    for i in range(np.shape(t1t5t6)[0]):
        T01 = DHTable2HomTrans([[np.pi/2,0,d1,t1t5t6[i][0]]])
        T56 = DHTable2HomTrans([[-np.pi/2,0,d5,t1t5t6[i][1]],[0,0,d6,t1t5t6[i][2]]])
        T3R = np.dot(np.dot(np.linalg.inv(T01),TUR5),np.linalg.inv(T56))
        
        theta234 = np.arctan2(T3R[1][0],T3R[0][0])
        theta23 = sol2Rik(T3R[0][3],T3R[1][3],a2,a3)
        for j in range(np.shape(theta23)[0]):
            theta4 = theta234 - theta23[j][0] - theta23[j][1]
            t2t3t4[k]=[theta23[j][0],theta23[j][1],theta4]
            k = k+1
    t1 = np.array([val for val in t1t5 for _ in (0, 1)])[:,0]
    t5 = np.array([val for val in t1t5 for _ in (0, 1)])[:,1]
    t6 = np.array([val for val in theta6 for _ in (0, 1)])
    ikUR5 = np.hstack((t1.reshape(8,1),t2t3t4,t5.reshape(8,1),t6.reshape(8,1)))
    return ikUR5

#conversion
def conversion(iks):
    iks_converted = []
    for i in range(len(iks)):
        b = []
        for j in range(len(iks[i])):
            if iks[i][j] > 0:
                if (iks[i][j])> math.pi:
                    iks[i][j] = iks[i][j] - 2*math.pi
            elif iks[i][j] < 0:
                if (-iks[i][j])> math.pi:
                    iks[i][j] = iks[i][j] + 2*math.pi
            else:
                iks[i][j] = iks[i][j]
            b.append(iks[i][j])
        iks_converted.append(b)
    return np.array(iks_converted)

In [1]:
def isRotationMatrix(R) :
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6

# Calculates rotation matrix to euler angles
# The result is the same as MATLAB except the order
# of the euler angles ( x and z are swapped ).
def rotationMatrixToEulerAngles(R) :
    assert(isRotationMatrix(R))
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
    singular = sy < 1e-6
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
    return np.array([x, y, z])

# RPY/Euler angles to Rotation Vector
def euler_to_rotVec(yaw, pitch, roll):
    # compute the rotation matrix
    Rmat = euler_to_rotMat(yaw, pitch, roll)
    
    theta = math.acos(((Rmat[0, 0] + Rmat[1, 1] + Rmat[2, 2]) - 1) / 2)
    sin_theta = math.sin(theta)
    if sin_theta == 0:
        rx, ry, rz = 0.0, 0.0, 0.0
    else:
        multi = 1 / (2 * math.sin(theta))
        rx = multi * (Rmat[2, 1] - Rmat[1, 2]) * theta
        ry = multi * (Rmat[0, 2] - Rmat[2, 0]) * theta
        rz = multi * (Rmat[1, 0] - Rmat[0, 1]) * theta
    return rx, ry, rz

def euler_to_rotMat(yaw, pitch, roll):
    Rz_yaw = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw),  np.cos(yaw), 0],
        [          0,            0, 1]])
    Ry_pitch = np.array([
        [ np.cos(pitch), 0, np.sin(pitch)],
        [             0, 1,             0],
        [-np.sin(pitch), 0, np.cos(pitch)]])
    Rx_roll = np.array([
        [1,            0,             0],
        [0, np.cos(roll), -np.sin(roll)],
        [0, np.sin(roll),  np.cos(roll)]])
    # R = RzRyRx
    rotMat = np.dot(Rz_yaw, np.dot(Ry_pitch, Rx_roll))
    return rotMat
